### 변동성 돌파 전략
- ETF 종목에서 많이 사용
- 변동폭 = 전일 고가 - 전일 저가 -> 목표가 =  당일 시가 + (변동폭 * K )

In [21]:
from pykrx import stock

In [22]:
df_kosdaq_150 = stock.get_market_ohlcv_by_date('20101001', '20241230', '229200')

In [23]:
df_kosdaq_150['변동폭'] = df_kosdaq_150['고가'] - df_kosdaq_150['저가']

In [24]:
df_kosdaq_150.columns

Index(['시가', '고가', '저가', '종가', '거래량', '등락률', '변동폭'], dtype='object')

In [25]:
df_kosdaq_150['전일변동폭'] = df_kosdaq_150['변동폭'].shift(periods=1)
df_kosdaq_150.head(4)

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭
날짜,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0


In [26]:
df_kosdaq_150

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭
날짜,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0
2015-10-07,10094,10094,9947,9949,21324,-2.230739,147,102.0
...,...,...,...,...,...,...,...,...
2024-12-20,11235,11255,10925,10975,9686866,-2.401067,330,180.0
2024-12-23,11110,11285,11000,11155,14527835,1.640091,285,330.0
2024-12-24,11205,11270,11135,11225,6202421,0.627521,135,285.0


In [27]:
df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * 0.5)
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭,목표가
날짜,,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0,10168.5
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0,10267.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0,10275.5
2015-10-07,10094,10094,9947,9949,21324,-2.230739,147,102.0,10145.0


In [28]:
df_kosdaq_150['종가'] / df_kosdaq_150['목표가']

날짜
2015-10-01         NaN
2015-10-02    0.997984
2015-10-05    0.996494
2015-10-06    0.990317
2015-10-07    0.980680
                ...   
2024-12-20    0.969095
2024-12-23    0.989357
2024-12-24    0.989205
2024-12-26    0.983506
2024-12-27    0.981996
Length: 2272, dtype: float64

In [29]:
conditions = df_kosdaq_150['고가'] >= df_kosdaq_150['목표가']     #매도 조건
conditions

날짜
2015-10-01    False
2015-10-02    False
2015-10-05    False
2015-10-06    False
2015-10-07    False
              ...  
2024-12-20    False
2024-12-23     True
2024-12-24    False
2024-12-26    False
2024-12-27     True
Length: 2272, dtype: bool

In [30]:
import numpy as np
np.where(conditions, df_kosdaq_150['종가']  / df_kosdaq_150['목표가'], 1)
df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가']  / df_kosdaq_150['목표가'], 1)

In [31]:
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭,목표가,수익률
날짜,,,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN,NaN,1.0
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0,10168.5,1.0
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0,10267.0,1.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0,10275.5,1.0
2015-10-07,10094,10094,9947,9949,21324,-2.230739,147,102.0,10145.0,1.0


In [32]:
df_kosdaq_150['수익률'].cumprod()

날짜
2015-10-01    1.000000
2015-10-02    1.000000
2015-10-05    1.000000
2015-10-06    1.000000
2015-10-07    1.000000
                ...   
2024-12-20    2.774530
2024-12-23    2.745001
2024-12-24    2.745001
2024-12-26    2.745001
2024-12-27    2.695580
Name: 수익률, Length: 2272, dtype: float64

#### 누적 수익률 이용한 CAGR

In [33]:
delta_days = df_kosdaq_150.index[-1] - df_kosdaq_150.index[0]
delta_days.days

3375

In [34]:
year = delta_days.days / 365

In [ ]:
#CAGR
CAGR = df_kosdaq_150['누적수익률'].iloc[-1] ** (1/year) - 1
CAGR * 100

KeyError: '누적 수익률'

### 전략 적용 vs 단순 보유

In [41]:
import matplotlib.pyplot as plt

In [44]:

#변동성 돌파 전략
plt.plot(df_kosdaq_150['누적수익률'], label='volatility rate')

#단순 보유
plt.plot(df_kosdaq_150['종가'] / df_kosdaq_150['종가'].iloc[0])

plt.grid()
plt.legend()
plt.show()

KeyError: '누적수익률'